In [172]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [173]:
uw_path = r"../userfiles/UW_Churn_Pred_Data.xls"

# List all sheet names
xls = pd.ExcelFile(uw_path)
print("Sheets in UW_Churn_Pred_Data.xls:", xls.sheet_names)

# Load a specific sheet (replace 'Sheet1' with the correct name)
df_uw = pd.read_excel(uw_path, sheet_name=xls.sheet_names[0])
print("Shape of loaded sheet:", df_uw.shape)

# Optional: show first 5 rows
df_uw.head()

Sheets in UW_Churn_Pred_Data.xls: ['Data', 'Legend', 'N10', 'B30 Pro', 'Data Before Feb 13']
Shape of loaded sheet: (4349, 28)


,Sale Channel,Model,Warranty,Feedback,Verification,Defect / Damage type,Responsible Party,Type,Spare Parts Used if returned,Final Status,...,Bluetooth (# of pairs),Wifi/Internet Connection,Wallpaper,Registered Email,last boot - activate,last boot - interval,interval date,last bootl date,activate date,Age Range
0,B2C 3rd party,B20,Yes,Amazon Return,"no damages, no issues",No defect,Customer: Remorse,Return,No,Refurb A,...,1,True,1,1.0,0.000000,0.000000,2024-11-18 00:52:52,2024-11-18 00:52:52,2024-11-18 00:52:52,30-40
1,B2C 3rd party,A23 Plus,Yes,Amazon Return,"no damages, no issues",No defect,Customer: Remorse,Return,No,Refurb B,...,0,True,1,1.0,0.000023,0.000000,2024-11-18 00:35:59,2024-11-18 00:35:59,2024-11-18 00:35:57,30-40
2,B2C 3rd party,A15,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5,True,2,NaN,0.000000,0.000000,2024-11-17 23:42:12,2024-11-17 23:42:12,2024-11-17 23:42:12,20-30
3,B2C 3rd party,A23 Plus,Yes,Amazon Return,"no damages, no issues",No defect,Customer: Remorse,Return,No,Refurb B,...,0,True,1,0.0,0.151493,0.151481,2024-11-17 23:28:58,2024-11-18 03:07:06,2024-11-17 23:28:57,20-30
4,B2C 3rd party,B20,Yes,Amazon Return,"no damages, no issues",No defect,Customer: Remorse,Return,No,Refurb B,...,1,True,0,1.0,0.000000,0.000000,2024-11-17 21:50:16,2024-11-17 21:50:16,2024-11-17 21:50:16,40-50


In [174]:
sheets_with_churn = ["N10", "B30 Pro", "Data Before Feb 13"]

dfs = {s: pd.read_excel(uw_path, sheet_name=s) for s in sheets_with_churn}

churn_cols = ['Chrn Flag', 'Churn', 'Churn Flag']

for name, df in dfs.items():
    # Find the churn column in this sheet
    for col in churn_cols:
        if col in df.columns:
            df['Churn'] = df[col]  # unify name
            break
    # Drop original churn-like columns
    for col in churn_cols:
        if col in df.columns and col != 'Churn':
            df.drop(columns=col, inplace=True)

In [175]:
for name in sheets_with_churn:
    df = pd.read_excel(uw_path, sheet_name=name)
    print(f"{name}: {df.shape[0]} rows")


N10: 970 rows
B30 Pro: 3142 rows
Data Before Feb 13: 4995 rows


In [176]:
for name, df in dfs.items():
    if 'Return date' in df.columns and 'active_date' in df.columns:
        # Ensure datetime type
        print(df['Return date'].head(10))
        df['Return date'] = pd.to_datetime(df['Return date'], errors='coerce')
        df['active_date'] = pd.to_datetime(df['active_date'], errors='coerce')

        # Compute return - activate in days
        df['return - activate'] = (df['Return date'] - df['active_date']).dt.total_seconds() / (3600*24)
    else:
        df['return - activate'] = None  # if either column missing

0   2024-11-26 08:10:00
1   2024-12-09 15:50:00
2   2024-12-09 15:50:00
3   2024-12-09 08:00:00
4   2024-12-09 08:00:00
5   2024-12-02 07:50:00
6   2024-12-02 14:40:00
7   2024-12-09 15:50:00
8   2024-12-12 07:40:00
9   2024-12-12 07:40:00
Name: Return date, dtype: datetime64[ns]
0                No Data
1                No Data
2                No Data
3                No Data
4                No Data
5                No Data
6                No Data
7    2024-09-30 07:50:00
8    2024-09-30 07:50:00
9                No Data
Name: Return date, dtype: object


C:\Users\liawi\AppData\Local\Temp\ipykernel_17752\2872875428.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Return date'] = pd.to_datetime(df['Return date'], errors='coerce')


In [177]:
for name, df in dfs.items():
    if 'sim_info' in df.columns:
        df['sim_info'] = df['sim_info'].apply(lambda x: 'inserted' if x != 'uninserted' else 'uninserted')
        df['sim_info'] = df['sim_info'].astype(str)

In [178]:
# Get column sets for N10 and B30 Pro
cols_n10 = set(dfs['N10'].columns)
cols_b30 = set(dfs['B30 Pro'].columns)

# Columns present in both
common_cols = cols_n10.intersection(cols_b30)
print("Columns present in BOTH N10 and B30 Pro:")
print(sorted(common_cols))

# Columns NOT in both (i.e., appear in only one of them)
not_in_both = cols_n10.symmetric_difference(cols_b30)
print("\nColumns NOT present in both (only in one sheet):")
print(sorted(not_in_both))


Columns present in BOTH N10 and B30 Pro:
['Analysis and Verification', 'Channel', 'Churn', 'Defect / Damage type', 'Final Status', 'Responsible Party', 'Return date', 'Spare Parts Usage', 'Type', 'Warranty', 'active_date', 'interval_date', 'last boot - active', 'last boot - interval', 'last_boot_date', 'model', 'register_email', 'return - activate', 'sim_info']

Columns NOT present in both (only in one sheet):
['activate']


In [179]:
combined_df = pd.concat(dfs.values(), ignore_index=True)

# Find columns with both Churn = 0 and Churn = 1
cols_with_both_churn = []
for col in common_cols:
    if combined_df.groupby(col)['Churn'].nunique().max() == 2:
        cols_with_both_churn.append(col)

print("Columns with data for both Churn = 0 and Churn = 1:")
print(cols_with_both_churn)

Columns with data for both Churn = 0 and Churn = 1:
['Return date', 'Channel', 'Spare Parts Usage', 'last boot - interval', 'interval_date', 'Warranty', 'Responsible Party', 'last_boot_date', 'sim_info', 'Analysis and Verification', 'register_email', 'Defect / Damage type', 'active_date', 'last boot - active', 'Type', 'model', 'Final Status']


C:\Users\liawi\AppData\Local\Temp\ipykernel_17752\2105686403.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dfs.values(), ignore_index=True)


In [180]:
drop_cols = ['activate', 'model', 'Office Time In', 'Office Date', 'Month', 
             'Product/Model #', 'Device number', 'Spare Parts Usage', 'Source',
             'Final Status','Analysis and Verification', 'Defect / Damage type',
             'Responsible Party', 'promotion_email', 'Channel', 'Warranty', 'Type']
combined_df = combined_df.drop(columns=drop_cols)


In [181]:
# Separate by Churn
churn_1 = combined_df[combined_df['Churn'] == 1].head()
churn_0 = combined_df[combined_df['Churn'] == 0].head()

# Concatenate for a quick preview
sample_head = pd.concat([churn_1, churn_0], ignore_index=True)
print(sample_head)


     sim_info register_email        interval_date       last_boot_date  \
0    inserted            1.0  2024-10-28 01:19:01  2024-10-28 01:19:01   
1    inserted            1.0  2024-11-12 07:18:05  2024-11-01 23:32:12   
2  uninserted            1.0  2024-11-01 15:52:34  2024-11-01 15:52:34   
3    inserted            1.0  2024-11-11 05:10:26  2024-11-10 04:30:33   
4    inserted            1.0  2024-11-04 01:37:10  2024-11-04 01:48:00   
5    inserted            1.0  2024-12-12 17:32:34  2024-12-05 14:33:21   
6    inserted            1.0  2024-12-12 14:18:42  2024-12-12 17:19:19   
7    inserted            1.0  2024-11-10 00:55:11  2024-11-15 07:33:28   
8    inserted            1.0  2024-10-27 23:55:22  2024-10-18 19:02:56   
9    inserted            1.0  2024-12-12 23:23:00  2024-11-27 22:17:54   

           active_date  last boot - interval  last boot - active  \
0  2024-10-28 01:19:00              0.000000            0.000012   
1  2024-10-31 22:29:14            -10.323530     

In [193]:
df.groupby("Churn")["register_email"].value_counts(normalize=True)


Churn  register_email
0.0    1.0               0.666667
       0.0               0.333333
1.0    0.0               0.577181
       1.0               0.422819
Name: proportion, dtype: float64

In [190]:
cols = [
    "last boot - interval",
    "last boot - active",
    "return - activate"
]

summary = (
    combined_df.groupby("Churn")[cols]
      .describe(percentiles=[0.25, 0.5, 0.75])
)

summary


last boot - interval                                              \
                     count       mean        std         min       25%   
Churn                                                                    
0.0                 3072.0  -5.286295  16.566998 -113.939317 -9.919905   
1.0                   86.0  11.377557  13.305775  -10.323530  0.000000   

                                        last boot - active             ...  \
             50%        75%         max              count       mean  ...   
Churn                                                                  ...   
0.0    -2.176516   0.153883  113.381921             3072.0  56.229436  ...   
1.0    14.143235  21.390148   39.992176               86.0  14.406895  ...   

                              return - activate                        \
              75%         max             count       mean        std   
Churn                                                                   
0.0    104.348568  139.369063               8.0  30.482335  40.917622   
1.0     23.032222   39.992188              42.0  27.803207   9.624106   

                                                              
             min        25%        50%        75%        max  
Churn                                                         
0.0   -10.614734  -0.529523  14.408848  59.905668  99.806806  
1.0     4.935428  24.597593  28.363843  34.359873  49.755208  

[2 rows x 24 columns]

In [182]:
# Combine to see all columns at once (for comparison only)
all_columns = sorted(set().union(*[df.columns for df in dfs.values()]))
print(f"All unique columns across sheets: {len(all_columns)}")

# Column presence across sheets
presence = pd.DataFrame(index=all_columns)
for name, df in dfs.items():
    presence[name] = presence.index.isin(df.columns)

presence["present_in_all"] = presence[sheets_with_churn].all(axis=1)
presence["present_in_any"] = presence[sheets_with_churn].any(axis=1)

print("Columns present in all sheets:")
print(presence[presence["present_in_all"]].index.tolist())

print("\nColumns present in only some sheets:")
print(presence[(presence["present_in_all"]==False) & (presence["present_in_any"])].index.tolist())


All unique columns across sheets: 27
Columns present in all sheets:
['Churn', 'Defect / Damage type', 'Final Status', 'Responsible Party', 'Type', 'Warranty', 'active_date', 'interval_date', 'last_boot_date', 'register_email', 'return - activate', 'sim_info']

Columns present in only some sheets:
['Analysis and Verification', 'Channel', 'Device number', 'Month', 'Office Date', 'Office Time In', 'Product/Model #', 'Return date', 'Source', 'Spare Parts Usage', 'activate', 'last boot - active', 'last boot - interval', 'model', 'promotion_email']


In [183]:
# Missing value stats per sheet
missing_stats = {}
for name, df in dfs.items():
    missing_stats[name] = df.isna().mean().sort_values(ascending=False)

missing_df = pd.concat(missing_stats, axis=1)
print("\nTop missing values per sheet:")
display(missing_df.head(20))


Top missing values per sheet:


,N10,B30 Pro,Data Before Feb 13
Spare Parts Usage,0.983505,0.000000,NaN
Return date,0.983505,0.989179,NaN
Type,0.983505,0.000000,0.946146
Channel,0.983505,0.000000,NaN
Churn,0.983505,0.000000,0.945145
Final Status,0.983505,0.000000,0.947347
return - activate,0.983505,0.989179,1.000000
Warranty,0.983505,0.000000,0.947147
Responsible Party,0.983505,0.000000,0.947147
Defect / Damage type,0.983505,0.000000,0.947147


In [184]:
# Column types per sheet
dtype_map = pd.DataFrame(index=all_columns)
for name, df in dfs.items():
    dtype_map[name] = df.dtypes.astype(str)

print("\nColumn types per sheet:")
display(dtype_map)


Column types per sheet:


,N10,B30 Pro,Data Before Feb 13
Analysis and Verification,object,object,NaN
Channel,object,object,NaN
Churn,float64,int64,float64
Defect / Damage type,object,object,object
Device number,NaN,NaN,int64
Final Status,object,object,object
Month,NaN,NaN,object
Office Date,NaN,NaN,datetime64[ns]
Office Time In,NaN,NaN,object
Product/Model #,NaN,NaN,object


In [185]:
# Cardinality of categorical columns
cardinality = {}
for name, df in dfs.items():
    cat_cols = df.select_dtypes(include='object').columns
    card = df[cat_cols].nunique().sort_values(ascending=False)
    cardinality[name] = card

print("\nTop categorical column cardinalities per sheet:")
for name, s in cardinality.items():
    print(f"\n{name}:")
    display(s.head(10))


Top categorical column cardinalities per sheet:

N10:


last_boot_date               969
interval_date                966
sim_info                       2
Final Status                   2
model                          1
Type                           1
Channel                        1
Warranty                       1
Analysis and Verification      1
Defect / Damage type           1
dtype: int64


B30 Pro:


last_boot_date               3136
interval_date                3102
Analysis and Verification       7
Defect / Damage type            6
Final Status                    6
Responsible Party               4
Channel                         3
Type                            3
Spare Parts Usage               3
sim_info                        2
dtype: int64


Data Before Feb 13:


active_date             4882
last_boot_date          4874
interval_date           4842
Office Time In            39
Defect / Damage type      13
Month                     11
Final Status              10
Responsible Party          5
Product/Model #            3
Source                     2
dtype: int64

In [186]:
for name, df in dfs.items():
    if 'sim_info' in df.columns:
        print(f"\nSheet: {name}")
        print(df['sim_info'].describe())
        print("Unique values:", df['sim_info'].nunique())
        display(df['sim_info'].head(10))



Sheet: N10
count          970
unique           2
top       inserted
freq           637
Name: sim_info, dtype: object
Unique values: 2


0      inserted
1      inserted
2    uninserted
3      inserted
4      inserted
5    uninserted
6    uninserted
7      inserted
8      inserted
9      inserted
Name: sim_info, dtype: object


Sheet: B30 Pro
count         3142
unique           2
top       inserted
freq          2705
Name: sim_info, dtype: object
Unique values: 2


0    uninserted
1      inserted
2      inserted
3      inserted
4      inserted
5      inserted
6      inserted
7      inserted
8    uninserted
9      inserted
Name: sim_info, dtype: object


Sheet: Data Before Feb 13
count         4995
unique           2
top       inserted
freq          3998
Name: sim_info, dtype: object
Unique values: 2


0      inserted
1    uninserted
2      inserted
3      inserted
4      inserted
5      inserted
6    uninserted
7    uninserted
8      inserted
9      inserted
Name: sim_info, dtype: object